In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
import numpy as np
import AlgoPlotting as plt
import itertools as it

In [ ]:
def shape_it(X):
    return np.expand_dims(X.reshape((-1,1)),2)

In [ ]:
def generate_prediction_data(price_data, sell_target_diff=1.00, stop_loss_diff=1.00, commission=0):
    predictions = []
    N = len(price_data)
    for ii in range(N):
        sell_target = price_data[ii] + sell_target_diff
        stop_loss = price_data[ii] - stop_loss_diff
        jj = ii
        while price_data[jj] >= stop_loss and price_data[jj] <= sell_target and jj < N-1:
            if jj < N-1:
                jj += 1
        if price_data[jj] >= sell_target:
            prediction = 1
        else: # price_data[jj] <= stop_loss:
            prediction = 0
#         elif jj >= N and price_data[N-1] > price_data[ii]:
#             prediction = price_data[N-1]/sell_target
#         else:
#             prediction = -stop_loss/price_data[N-1]
        
        predictions.append(prediction)
    return np.array(predictions)
        

In [ ]:
# This method is used to filter out spikes one way or the other in the bullish/bearish classification data.
def filter_prediction_data(prediction_data, tol=2, use_uncertain_class=False):
    # Get array of values and lengths of sequences in prediction_data (e.g. [0 0 0 1 1] = [(0, 3), (2, 1)])
    sequences = np.array([(k, sum(1 for _ in i)) for k, i in it.groupby(prediction_data)])
    
    # Split up and get indexing vectors
    values = sequences[:,0]
    lengths = sequences[:,1]
    ends = np.cumsum(lengths)
    starts = np.hstack((0, ends[:-1]))
    
    # Use uncertain classifier if specified, otherwise treat as bearish
    bullish = 1
    bearish = 0
    if use_uncertain_class:
        uncertain = 0.5
    else:
        uncertain = bearish
    
    # Convert to float np array, otherwise assigning to 0.5 results in 0
    filtered_prediction_data = np.array(prediction_data).astype(float)
    N = len(values)
    for ii in range(N):
        # Check to see if the sequence is too short and needs to be removed
        if lengths[ii] < tol:
            if ii == 0:
                if lengths[1] >= tol:
                    value = values[1]
                else:
                    value = uncertain
            elif ii == N-1:
                if lengths[N-2] >= tol:
                    value = values[N-2]
                else:
                    value = uncertain
            elif values[ii-1] == values[ii+1] and lengths[ii-1] >= tol and lengths[ii+1] >= tol:
                    value = values[ii-1]
            else:
                value = uncertain
            #print(value)
            #print(filtered_prediction_data)
            filtered_prediction_data[starts[ii]:ends[ii]] = value
            #print(filtered_prediction_data)
    return filtered_prediction_data